In [60]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import requests
import selenium

In [61]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.21M/6.21M [00:00<00:00, 7.05MB/s]


In [62]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [63]:
# set up html parser and parent element
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [64]:
slide_elem.find('div', class_='content_title')

<div class="content_title">3 Things We've Learned From NASA's Mars InSight </div>

In [65]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"3 Things We've Learned From NASA's Mars InSight "

In [66]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Scientists are finding new mysteries since the geophysics mission landed two years ago.'

## Featured Images -JPL space image

In [67]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [68]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [69]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [70]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [71]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

## Mars Facts

In [72]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [73]:
# Convert to html to put into a website
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [85]:
# 1. Use browser to visit the URL 
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

browser = init_browser()
url = 'https://marshemispheres.com/'
browser.visit(url)

In [90]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []


# 3. Write code to retrieve the image urls and titles for each hemisphere.

# 3b. Set up the soup object
html = browser.html
hemi_soup = soup(html, 'html.parser')

# 3b. Find (almost) all h3 tags
main = hemi_soup.find_all('h3')[0:4]

# 3d.  Iterate over the h3 tags in the soup object and pull out the titles and image links
for hemi in main:
    hemispheres = {}
    
    # Click on the link for each hemishphere
    page = browser.find_by_text(hemi.text)
    page.click()
    
    # Create a soup object for the new page
    html = browser.html
    new_soup = soup(html, 'html.parser')
    
    # Find the Div with class downloads
    section = new_soup.find('div', class_='downloads')
    
    # Find and pull the image and create a full URL for it
    img_section = section.find('a', target='_blank')
    href = img_section['href']
    img_url = f'https://marshemispheres.com/{href}'
    
    # Find and pull the title
    pretitle = new_soup.find('div', class_='cover')
    title = pretitle.find('h2').get_text()
    
    # Add the img and url to the dictionary
    hemispheres['URL'] = img_url
    hemispheres['Title'] = title
    
    # Append the dict to the list
    hemisphere_image_urls.append(hemispheres)
    
    # Go back to the original page
    browser.back()








    
    
    

In [91]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'URL': 'https://marshemispheres.com/images/full.jpg',
  'Title': 'Cerberus Hemisphere Enhanced'},
 {'URL': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'Title': 'Schiaparelli Hemisphere Enhanced'},
 {'URL': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'Title': 'Syrtis Major Hemisphere Enhanced'},
 {'URL': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'Title': 'Valles Marineris Hemisphere Enhanced'}]

In [92]:
browser.quit()